In [ ]:
# Automatic Number Plate Recognition (ANPR) using CNN
# Author: Urmi Ganguly
# Description: This script performs number plate recognition using OpenCV and a custom CNN.

import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

# -------------------------------
# TensorFlow Import with Error Handling
# -------------------------------
try:
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
    from tensorflow.keras.preprocessing.image import ImageDataGenerator
    from tensorflow.keras.models import load_model
    from tensorflow.keras.preprocessing.image import img_to_array
except ImportError as e:
    print("TensorFlow is not installed or not configured correctly.")
    print("Error:", e)
    print("Please ensure TensorFlow is installed and compatible with your Python version.")
    exit(1)

# -------------------------------
# 1. User Input for Paths
# -------------------------------
try:
    train_dir = input("Enter training data path: ").strip().strip('"').strip("'")
    val_dir = input("Enter validation data path: ").strip().strip('"').strip("'")
except Exception as e:
    print("Error reading input paths:", e)
    exit(1)

# -------------------------------
# 2. Image Generators
# -------------------------------
img_width, img_height = 50, 50
batch_size = 32

try:
    datagen = ImageDataGenerator(rescale=1./255)
    train_generator = datagen.flow_from_directory(
        train_dir,
        target_size=(img_width, img_height),
        color_mode='grayscale',
        batch_size=batch_size,
        class_mode='categorical')

    val_generator = datagen.flow_from_directory(
        val_dir,
        target_size=(img_width, img_height),
        color_mode='grayscale',
        batch_size=batch_size,
        class_mode='categorical')
except Exception as e:
    print("Error loading image data:", e)
    exit(1)

# -------------------------------
# 3. CNN Model
# -------------------------------
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 1)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(train_generator.class_indices), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# -------------------------------
# 4. Model Training
# -------------------------------
epochs = 10
model.fit(train_generator, validation_data=val_generator, epochs=epochs)
model.save("character_recognition_cnn.keras", save_format="keras")

# -------------------------------
# 5. Plate Detection
# -------------------------------
def detect_plate(image_path):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    filtered = cv2.bilateralFilter(gray, 11, 17, 17)
    edged = cv2.Canny(filtered, 30, 200)
    contours, _ = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)[:10]

    for c in contours:
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.018 * peri, True)
        if len(approx) == 4:
            x, y, w, h = cv2.boundingRect(approx)
            plate = image[y:y+h, x:x+w]
            cv2.imshow("Detected Plate", plate)
            cv2.waitKey(0)
            cv2.destroyAllWindows()
            return plate
    return None

# -------------------------------
# 6. Character Segmentation
# -------------------------------
def segment_characters(plate_img):
    gray = cv2.cvtColor(plate_img, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV)
    contours, _ = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
   
    characters = []
    for i, cnt in enumerate(sorted(contours, key=lambda ctr: cv2.boundingRect(ctr)[0])):
        x, y, w, h = cv2.boundingRect(cnt)
        if h / plate_img.shape[0] >= 0.5 and w / plate_img.shape[1] <= 0.5:
            char = thresh[y:y+h, x:x+w]
            resized = cv2.resize(char, (img_width, img_height))
            characters.append(resized)

            # Debug display
            cv2.imshow(f"Character {i+1}", resized)
            cv2.waitKey(0)
            cv2.destroyAllWindows()
    return characters

# -------------------------------
# 7. Character Recognition
# -------------------------------
def recognize_characters(characters, model, class_labels):
    result = ""
    for char in characters:
        char = char.astype("float32") / 255.0
        char = np.expand_dims(char, axis=-1)
        char = np.expand_dims(char, axis=0)
        pred = model.predict(char, verbose=0)
        label = class_labels[np.argmax(pred)]
        result += label
    return result

# -------------------------------
# 8. Predict Plate Number
# -------------------------------
def predict_plate_number(image_path, model, class_indices):
    plate_img = detect_plate(image_path)
    if plate_img is not None:
        chars = segment_characters(plate_img)
        labels_map = {v: k.split("_")[1] for k, v in class_indices.items()}
        return recognize_characters(chars, model, labels_map)
    else:
        return "Plate not detected"

# -------------------------------
# 9. User Testing
# -------------------------------
test_image_path = input("Enter path to a test image: ").strip().strip('"').strip("'")

if os.path.exists(test_image_path):
    model = load_model("character_recognition_cnn.keras")
    predicted_plate = predict_plate_number(test_image_path, model, train_generator.class_indices)
    print("Detected Plate Number:", predicted_plate)

else:
    print("Test image path is invalid or does not exist.")



Enter training data path: C:\Users\URMI GANGULY\Documents\IITG\Automatic Number Plate Detection\data\data\train
Enter validation data path: C:\Users\URMI GANGULY\Documents\IITG\Automatic Number Plate Detection\data\data\val
Found 864 images belonging to 36 classes.
Found 216 images belonging to 36 classes.



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 48, 48, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 24, 24, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 22, 22, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 11, 11, 64)        0         
 g2D)   